# Model_Evaluation_and_Metrics.ipynb

本 Notebook 將介紹機器學習中模型評估與指標計算的常見手法與方法，包括：

1. 資料集劃分策略：
   - 訓練集/驗證集/測試集
   - K-Fold Cross-Validation
   - 留一法 (Leave-One-Out)
   - Stratified 分層取樣 (針對分類)

2. 分類指標：
   - Accuracy
   - Precision, Recall, F1 Score
   - 混淆矩陣 (Confusion Matrix)
   - ROC 曲線、AUC

3. 回歸指標：
   - MSE (Mean Squared Error)
   - MAE (Mean Absolute Error)
   - R² (Coefficient of Determination)

4. Cross-Validation、Learning Curve 與 Validation Curve 示範：
   - 使用 `cross_val_score` 評估穩定性
   - Learning Curve 觀察隨訓練數量變化的表現
   - Validation Curve 觀察參數變化對性能的影響

透過本範例，您將了解在模型訓練與選擇過程中如何有效地評估模型，並選擇合適的指標判斷模型性能。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.2)

from sklearn.datasets import load_iris, make_classification, make_regression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, learning_curve, validation_curve
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,
                             roc_curve, auc, mean_squared_error, mean_absolute_error, r2_score, classification_report)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler


## 1. 訓練/驗證/測試集劃分

一般流程：
- 訓練集(Training Set): 用於訓練模型
- 驗證集(Validation Set): 用於模型選擇與調參
- 測試集(Test Set): 最終評估模型泛化性能

簡單範例使用 Iris 資料集進行分類，並將資料分為訓練與測試集。

In [ ]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

print("訓練集大小:", X_train_i.shape, " 測試集大小:", X_test_i.shape)

### Cross-Validation (K-Fold)
透過 K 折交叉驗證來評估模型穩定性，降低對於單一劃分的偶然性影響。

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
scores = cross_val_score(clf, X_iris, y_iris, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", scores)
print("Mean CV Accuracy:", np.mean(scores))

### StratifiedKFold
用於分類問題，以確保每折中的類別比例與原始資料相近。

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_skf = cross_val_score(clf, X_iris, y_iris, cv=skf, scoring='accuracy')
print("StratifiedKFold CV Scores:", scores_skf)
print("Mean CV Accuracy:", np.mean(scores_skf))

## 2. 分類評估指標

### Accuracy
整體正確率，對類別分布不平衡的情況不敏感。

### Precision, Recall, F1 Score
適用於不平衡類別問題。
- Precision (精確率): 在預測為正的樣本中，有多少是真的正。
- Recall (召回率): 在實際為正的樣本中，有多少被正確預測為正。
- F1 Score: Precision 與 Recall 的調和平均。

### Confusion Matrix
顯示各類別的預測與實際情況。

#### 示範：使用合成資料進行二元分類並計算上述指標

In [ ]:
X_bin, y_bin = make_classification(n_samples=500, n_features=5, n_informative=3, 
                                 n_redundant=1, n_clusters_per_class=1, random_state=42)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_bin, y_bin, test_size=0.3, random_state=42)

clf_bin = DecisionTreeClassifier(random_state=42)
clf_bin.fit(X_train_b, y_train_b)
y_pred_b = clf_bin.predict(X_test_b)

acc_b = accuracy_score(y_test_b, y_pred_b)
prec_b = precision_score(y_test_b, y_pred_b)
rec_b = recall_score(y_test_b, y_pred_b)
f1_b = f1_score(y_test_b, y_pred_b)

print("Accuracy:", acc_b)
print("Precision:", prec_b)
print("Recall:", rec_b)
print("F1 Score:", f1_b)
print("\nClassification Report:\n", classification_report(y_test_b, y_pred_b))

cm_b = confusion_matrix(y_test_b, y_pred_b)
sns.heatmap(cm_b, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

### ROC曲線與AUC
ROC曲線顯示在不同分類閾值下的TPR(Recall)與FPR，AUC則是曲線下的面積。

In [ ]:
from sklearn.metrics import roc_curve, auc

# 必須使用 predict_proba 或 decision_function 產生分數
y_prob_b = clf_bin.predict_proba(X_test_b)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_b, y_prob_b)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

## 3. 回歸評估指標

### MSE (Mean Squared Error)
平方誤差平均，對大誤差較敏感。

### MAE (Mean Absolute Error)
平均絕對誤差。

### R² (Coefficient of Determination)
衡量模型對資料變異的解釋程度，越接近1越好。

#### 示範：使用合成回歸資料並計算上述指標

In [ ]:
X_reg, y_reg = make_regression(n_samples=200, n_features=3, noise=10, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

lr_reg = LinearRegression()
lr_reg.fit(X_train_r, y_train_r)
y_pred_r = lr_reg.predict(X_test_r)

mse_r = mean_squared_error(y_test_r, y_pred_r)
mae_r = mean_absolute_error(y_test_r, y_pred_r)
r2_r = r2_score(y_test_r, y_pred_r)

print("MSE:", mse_r)
print("MAE:", mae_r)
print("R²:", r2_r)

## 4. Learning Curve 與 Validation Curve

**Learning Curve**：顯示隨著訓練資料量增加，模型的訓練與驗證分數變化，判斷模型是否過擬合/欠擬合。

**Validation Curve**：顯示隨著某個模型參數的變化，訓練與驗證分數的變化，用於選擇較佳參數範圍。

In [ ]:
from sklearn.model_selection import learning_curve, validation_curve

# 以 LogisticRegression 舉例示範 Learning Curve
clf_lc = LogisticRegression(max_iter=1000)
train_sizes, train_scores, val_scores = learning_curve(clf_lc, X_iris, y_iris, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 5))

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, 'o-', color='r', label='Training score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='r')
plt.plot(train_sizes, val_mean, 'o-', color='g', label='Validation score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='g')
plt.xlabel('Training examples')
plt.ylabel('Score')
plt.title('Learning Curve (LogisticRegression)')
plt.legend(loc='best')
plt.show()

### Validation Curve 範例
以 DecisionTreeClassifier 的 max_depth 為例，觀察不同深度下的訓練與驗證分數。

In [ ]:
param_range = np.arange(1, 11)
train_scores_v, val_scores_v = validation_curve(DecisionTreeClassifier(random_state=42), 
                                               X_iris, y_iris, param_name="max_depth", param_range=param_range,
                                               cv=5, scoring="accuracy")

train_mean_v = np.mean(train_scores_v, axis=1)
train_std_v = np.std(train_scores_v, axis=1)
val_mean_v = np.mean(val_scores_v, axis=1)
val_std_v = np.std(val_scores_v, axis=1)

plt.figure(figsize=(8,6))
plt.plot(param_range, train_mean_v, 'o-', color='r', label='Training score')
plt.fill_between(param_range, train_mean_v - train_std_v, train_mean_v + train_std_v, alpha=0.2, color='r')
plt.plot(param_range, val_mean_v, 'o-', color='g', label='Validation score')
plt.fill_between(param_range, val_mean_v - val_std_v, val_mean_v + val_std_v, alpha=0.2, color='g')
plt.xlabel('max_depth')
plt.ylabel('Score')
plt.title('Validation Curve (Decision Tree)')
plt.legend(loc='best')
plt.show()

## 實務建議

1. 始終將資料分為訓練/測試集以評估泛化性能。
2. 使用交叉驗證 (Cross-Validation) 確保結果穩定，減少對單一劃分的依賴。
3. 依據問題特性選擇合適指標：
   - 不平衡類別問題：使用 Precision, Recall, F1 Score, AUC。
   - 一般分類問題：Accuracy 可做為初步指標。
   - 回歸問題：MSE、MAE、R²。
4. 使用 Learning Curve 確認模型是否欠擬合或過擬合，是否需要更多資料。
5. 使用 Validation Curve 觀察超參數對模型性能的影響。

透過綜合使用以上方法與指標，可以對模型性能有更深入的理解，並據此做出更好的模型選擇與調優策略。